In [1]:
import requests
import pandas as pd
import datetime as dt
import numpy as np
from datetime import timedelta
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

BASE_URL = "https://api.the-odds-api.com/v3/odds/?apiKey={yourAPIKey}&sport=soccer&region=eu"
response = requests.get(BASE_URL)

s= list(response.json().values())[1]
df_list= []

## Pinnacle Odds vs. Everyone

In [ ]:

#Level2 - here you'll get columns for each game 
cols= list(s[1].keys())
df= pd.DataFrame(columns= cols)
dat= []
for element in s:
    thing= element.values()
    extra= dict(zip(cols,thing))
    dat.append(extra)
df= df.append(dat)
# Level 3 add the away_team
df['away_team']= 0
for x,i in enumerate(df['teams']):
    if i[0] in df['home_team'][x]:
        df['away_team'][x]= i[1]
    else:
        df['away_team'][x]= i[0]
#Level 4- convert dates 
def unix_conv(unix):
    return(dt.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%dT%H:%M:%SZ'))
    
df['time']= df['commence_time'].map(unix_conv)
df['time']= pd.to_datetime(df['time'], format= "%Y/%m/%d %H:%M")+ timedelta(hours=2)

#Level 5 - add the columns with the odds        
k=[]
for p in df['sites']:
    for i in np.arange(0,len(p)):
        z= list(p[i].values())
        t= z[1]
        k.append(t)
        
a_list= list(set(k))
b_list= ['H','D','A']
oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]
oo.sort()
df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)
df[oo]= df[oo].astype('float')

#Level 6 - fill the columns with values 
for name, data in df[oo].iteritems():
    for x,i in enumerate(df['sites']):
        for j in np.arange(0,len(i)):
            imp= list(i[j].values())[1]
            odds= list(list(i[j].values())[3].values())[0]
            if df['home_team'][x]==df['teams'][x][0]:
                order= [0,2,1]
                odds= [odds[i] for i in order]
            elif df['home_team'][x]== df['teams'][x][1]:
                order= [1,2,0]
                odds= [odds[i] for i in order]
            if imp in name and name.endswith('H'):
                df[name][x]= odds[0]
                break
            elif imp in name and name.endswith('D'):
                df[name][x]= odds[1]
                break
            elif imp in name and name.endswith('A'):
                df[name][x]= odds[2]
                break
            else:
                continue
                
# Level 7 Calculate each bookmaker's fair probability [but we only need Pinnacle Sports]

df['ProbsPinnacleH']= 1/df['PinnacleH']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleD']= 1/df['PinnacleD']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleA']= 1/df['PinnacleA']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])


# Level 8 - calculate max odds 
df['MaxOddsH']= df[[x for x in df[oo].columns if x.endswith('H')]].max(axis=1)
df['MaxOddsD']= df[[x for x in df[oo].columns if x.endswith('D')]].max(axis=1)
df['MaxOddsA']= df[[x for x in df[oo].columns if x.endswith('A')]].max(axis=1)

# Level 9 - calculate margin in relation to pinnacle sports probability 
df['MarginH'] = df.ProbsPinnacleH*(df.MaxOddsH-1)-(1-df.ProbsPinnacleH)
df['MarginD'] = df.ProbsPinnacleD*(df.MaxOddsD-1)-(1-df.ProbsPinnacleD)
df['MarginA'] = df.ProbsPinnacleA*(df.MaxOddsA-1)-(1-df.ProbsPinnacleA)

# Level 10 - get max bookies' name
maxBookieH= df[[x for x in df[oo].columns if x.endswith('H')]]
maxBookieD= df[[x for x in df[oo].columns if x.endswith('D')]]
maxBookieA= df[[x for x in df[oo].columns if x.endswith('A')]]
df['MaxBookieH']= 'none'
df['MaxBookieD']= 'none'
df['MaxBookieA']= 'none'


for name, data in maxBookieH.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsH'][i]:
            df['MaxBookieH'][i]= name
            
            
for name, data in maxBookieD.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsD'][i]:
            df['MaxBookieD'][i]= name
            
            
for name, data in maxBookieA.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsA'][i]:
            df['MaxBookieA'][i]= name
            

relevant= ['sport_nice', 'home_team', 'away_team', 'time', 'MaxOddsH','MaxOddsD','MaxOddsA','ProbsPinnacleH', 'ProbsPinnacleD','ProbsPinnacleA', 'MaxBookieH','MaxBookieD', 'MaxBookieA', 'MarginH','MarginD','MarginA']

df_regular= df[relevant].dropna(subset=['MarginH'])
df_regular.name='df_regular'
df_list.append(df_regular)